# PipelinePromptTemplate 解释

PipelinePromptTemplate 是 LangChain 核心提示模块中的一个类，用于构建复杂的提示模板。
> 它允许将多个**独立的 template** 构建成一个完整且复杂的 prompt，组合成一个管道，最终生成一个完整的提示字符串，这样可以提高独立 prompt 的复用性，进一步增强模块化带来的优势。


## 主要作用

- **组合子提示**：通过 `pipelinePrompts` 数组，将多个 PromptTemplate 实例组合在一起。
- **部分填充**：可以使用 `partial` 方法预填充某些变量，如日期函数。
- **最终合成**：使用 `finalPrompt` 来定义最终的提示结构，引用子提示的输出作为变量。

## 使用方法

1. 定义各个子提示模板 (PromptTemplate)。
2. 创建 PipelinePromptTemplate 实例，指定 `pipelinePrompts`（包含名称和对应提示）和 `finalPrompt`（最终模板）。
3. 调用 `format` 方法，传入所有必要的变量，生成最终提示。

在这个文件中，我们演示了如何使用 PipelinePromptTemplate 来创建一个智能管家的提示，结合日期、信息和任务，形成一个完整的上下文。

In [10]:
import {
  PromptTemplate,
  PipelinePromptTemplate,
} from "@langchain/core/prompts";

const getCurrentDateStr = () => {
  return new Date().toLocaleDateString();
};


const fullPrompt = PromptTemplate.fromTemplate(`
你是一个智能管家，今天是 {date}，你的主人的信息是{info1}, 
根据上下文，完成主人的需求
{task}`);

const datePrompt = PromptTemplate.fromTemplate("{date}，现在是 {period}");

const periodPrompt = await datePrompt.partial({
  date: getCurrentDateStr,
});
const infoPrompt = PromptTemplate.fromTemplate(
  "姓名是 {name}, 性别是 {gender}"
);

const taskPrompt = PromptTemplate.fromTemplate(`
我想吃 {period} 的 {food}。 
再重复一遍我的信息 {info2}`);

const composedPrompt = new PipelinePromptTemplate({
  pipelinePrompts: [
    {
      name: "date",
      prompt: periodPrompt,
    },
    {
      name: "info1",
      prompt: infoPrompt,
    },
    {
      name: "info2",
      prompt: infoPrompt,
    },
    {
      name: "task",
      prompt: taskPrompt,
    },
  ],
  finalPrompt: fullPrompt,
});

const formattedPrompt = await composedPrompt.format({
  period: "早上",
  name: "张三",
  gender: "male",
  food: "lemon",
});
console.log(formattedPrompt);


你是一个智能管家，今天是 2025/12/24，现在是 早上，你的主人的信息是姓名是 张三, 性别是 male, 
根据上下文，完成主人的需求

我想吃 早上 的 lemon。 
再重复一遍我的信息 姓名是 张三, 性别是 male
